# Practical GNN for Doctors
## How do I implement GNN?

### 들어가며

이번 시간에는 GNN을 실제로 어떻게 구현해보는지 알아보도록 하겠습니다. 제가 구현할 환경은 Python3에 설치된 [PyTorch](https://pytorch.org), [Deep Graph Library (DGL)](https://www.dgl.ai)를 이용할 예정이니 직접 구현을 함께 해보시기 위해서는 두 라이브러리를 설치하셔야 합니다. GPU는 따로 없어도 실험에 크게 무리는 없을 것으로 생각됩니다.
지난 시간에 언급되었던 것 처럼 이번 목표는 Graph Convolutional Network (GCN)을 구현하는 것입니다. 구현된 GCN을 이용하여 Pubmed 데이터셋의 노드 분류 태스크를 학습하고 그 성능을 확인해 보도록 하겠습니다. 먼저 필요한 라이브러리를 불러오겠습니다.

In [1]:
import torch
import torch.nn as nn
import dgl
import time

Using backend: pytorch


### 1. 데이터: Pubmed dataset
#### 데이터셋 불러오기
다음으로 모델을 구현하기 전 Pubmed 데이터셋을 불러오고 그 특징에 대해 둘러보도록 하겠습니다.
DGL에서는 대표적인 벤치마크 그래프 데이터셋들을 쉽게 사용할 수 있게 제공하고 있으며, Pubmed 데이터셋도 다음과 같이 불러올 수 있습니다.

In [2]:
dataset = dgl.data.PubmedGraphDataset()

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


#### 데이터셋 둘러보기
Pubmed 데이터셋은 당뇨와 관련된 Pubmed에 색인된 19,717개의 출판물을 하나의 노드로 보고, 각 출판물간의 인용관계를 엣지로 보는 인용네트워크(Citation network) 그래프입니다.
전체 데이터는 하나의 큰 그래프이며, 해당 그래프는 19,717개의 노드(NumNodes)와 88,651개의 엣지(NumEdges)로 이루어져 있습니다. 
하나의 노드 당 특징 벡터의 길이(NumFeats)는 500입니다. 
이 길이 500의 벡터의 원소는 각각 500개의 단어와 짝을 지어 해당 단어가 출판물(노드)에 나타나는지 여부를 빈도수에 맞추어 수치료 표현(TF-IDF: Text frequency - inverse document frequency)하게 됩니다. 
이 특징 벡터를 이용하면 각 노드에 어떠한 단어가 포함되어있는지 네트워크에 입력을 해 주는 것이 가능하게 됩니다.
이러한 그래프 구조와 노드 특징을 이용해서 이 출판물이 <DM, Experimental; DM, type 1; DM, type2>의 3개의 주제(NumClasses) 중 어떠한 주제에 속하는지 분류하는 것이 이번에 구현하는 GCN의 학습 목표가 됩니다.
데이터셋에는 그래프가 하나만 포함되어 있으므로 가장 첫 번째 그래프를 가져와 사용하도록 하겠습니다.

In [3]:
g = dataset[0]

print('=== 노드 관련 데이터들의 목록 ===')
print(g.ndata.keys())
print()

print('=== 노드 특징벡터 예시와 크기===')
print(g.ndata['feat'][:5])
print(g.ndata['feat'].shape)
print()

print('=== 노드의 라벨(정답) 예시와 크기=== ')
print(g.ndata['label'][:5])
print(g.ndata['label'].shape)
print()

print('=== 인접행렬 ===')
print(g.adjacency_matrix())

=== 노드 관련 데이터들의 목록 ===
dict_keys(['feat', 'label', 'test_mask', 'val_mask', 'train_mask'])

=== 노드 특징벡터 예시와 크기===
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0554, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
torch.Size([19717, 500])

=== 노드의 라벨(정답) 예시와 크기=== 
tensor([1, 1, 0, 2, 1])
torch.Size([19717])

=== 인접행렬 ===
tensor(indices=tensor([[    0,     0,     0,  ..., 19714, 19715, 19716],
                       [14442,  1378,  1544,  ..., 12278,  4284, 16030]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(19717, 19717), nnz=88651, layout=torch.sparse_coo)


DGL에서 제공하는 데이터셋의 그래프에는 노드와 관련하여 feat, label, train_mask, val_mask, test_mask를 제공하고 있음을 볼 수 있습니다.

##### 노드 특징
앞서 데이터셋을 둘러볼 때 이해했다시피 노드 특징벡터는 19717x500 크기의 행렬입니다.
각 행은 하나의 노드에 해당하고, 각 열은 특징벡터의 원소에 해당한다고 볼 수 있겠습니다.

##### 노드 라벨
각 노드에는 대응되는 라벨(정답)이 제공됩니다. 
이는 19717개의 벡터에 <0, 1, 2>중 하나의 값으로 표현이 되어 있습니다.

##### 인접행렬
마지막으로 인접행렬은 조금 특이한 구조인 것을 볼 수 있습니다.
indices, values, size, nnz, layout으로 나뉘어 행렬의 정보가 표시되어 있는데 layout을 가장 먼저 보겠습니다.
layout은 torch.sparse_coo로 되어있습니다.
이는 바로 인접행렬이 희소행렬(sparse matrix)이며 좌표 형식(coo: coordinate)으로 저장되어 있다는 뜻입니다.
인접행렬은 0과 1로 이루어져있는 노드개수x노드개수 크기의 행렬입니다 (위에 출력된 정보 중 size=(19717, 19717)입니다).
이는 노드 개수가 많아지면 메모리에 올려야 하는 행렬의 크기가 이에 제곱으로 비례해 커진다는 의미입니다.
여기에서 인접행렬이 희소행렬이라는 점을 이용하면 메모리 사용을 크게 줄일 수 있습니다.
희소행렬은 대부분의 성분이 0이고, 일부 성분이 0이 아닌 행렬을 의미합니다.
인접행렬의 특성상 0이 아닌 성분은 엣지의 개수와 같게 됩니다.
따라서 number of nonzero: nnz=88651 임을 볼 수 있습니다.
indices와 values는 coo형식의 sparse matrix에서 0이 아닌 성분의 좌표와 해당 좌표에서의 값을 각각 나타냅니다.
따라서 위에서 출력을 해 보지 않았지만 indices와 values는 각각 2x88651 (행렬좌표2종 x 엣지개수) 과 88651 (엣지개수)의 크기를 갖는 행렬/벡터일 것으로 예상할 수 있습니다.

### 2. 모델: Graph Convolution Network (GCN)
#### (1) 쉽게 구현하기: DGL에서 제공하는 GCN 구조를 이용

GCN은 GNN중 매우 기본이 되는 모델이기 때문에, DGL에서도 GCN 구조를 쉽게 불러올 수 있게 제공하고 있습니다.
GCN 층은 `dgl.nn.GraphConv` 로 불러올 수 있고, 이는 `torch.nn.module`을 상속받기 때문에 pytorch에서 모델을 만드는 방식을 그대로 적용하여 쉽게 모델을 구성할 수 있습니다.
다음과 같이 두 개의 층을 갖는 GCN 모델을 정의해보겠습니다.

In [4]:
class GCN1(nn.Module):
    def __init__(self, node_feat_length, num_hidden, num_classes):
        super().__init__()
        self.conv1 = dgl.nn.GraphConv(node_feat_length, num_hidden) # dgl.nn.GraphConv를 이용해 하나의 GCN 층을 구성할 수 있음.
        self.relu = nn.ReLU()
        self.conv2 = dgl.nn.GraphConv(num_hidden, num_classes)
        
    def forward(self, g, node_feat):
        h = self.conv1(g, node_feat)
        h = self.relu(h)
        h = self.conv2(g, h)
        return h

위와같이 미리 정의된 GCN 연산인 `dgl.nn.GraphConv`를 불러와 두 개의 층을 갖는 `GCN1` 모델을 쉽게 정의하였습니다.
pytorch에서 `nn.Module`을 상속받아 모델을 만드는 방법에 대해서는 이 강의에서 다루지는 않겠습니다. 
만약 pytorch 사용의 기본적인 부분에 대해서 이해가 필요하다면 [pytorch 공식 페이지의 tutorial](https://pytorch.org/tutorials/)을 먼저 이해하고 오시면 도움이 될 것입니다.

#### (2) 직접 구현하기: DGL에서 제공하는 Message-passing을 이용하여 GCN 구조를 직접 정의

우리는 GCN 연산에 대해 이해를 하였기 때문에 이를 직접 구현할 수도 있을 것입니다.
이를 위해서 `dgl.nn.GraphConv`의 역할을 하는 클래스인 `GraphConv`를 직접 정의하여 모델을 만들어 보도록 하겠습니다.
그 전에, DGL에서 제공하는 Message-passing 개념을 먼저 이해할 필요가 있습니다.

##### Message-passing
지난 시간에 Message-passing이라는 용어에 대해 간략히 언급한 적이 있습니다.

>AGGREGATE와 COMBINE은 GNN의 가장 핵심이 되는 부분으로 CNN으로 따지면 컨볼루션(Convolution)에 해당되는 부분입니다. 이 과정에서는 각 노드를 기준으로 연결이 존재하는 주변 노드들의 특징을 종합하게 되어 Message-passing이라고도 불립니다.

즉 DGL에서 Message-passing 개념을 제공한다는 의미는, AGGREGATE와 COMBINE함수를 원하는 대로 만들어서 GNN층을 만들 수 있다는 것과 비슷한 의미입니다.
구체적으로 Message-passing 개념을 바탕으로 GNN의 AGGREGATE/COMBINE을 다시 이해해보겠습니다.
Message-passing은 message를 만드는 과정과, 이를 passing하는 과정으로 나뉘어집니다. (각각 AGGREGATE, 그리고 COMBINE과 비슷한 과정입니다.)
Message를 만드는 과정은 그래프의 모든 엣지를 기준으로 진행이 되고, 이를 passing하는 과정은 그래프의 모든 노드를 기준으로 진행이 됩니다.
먼저 message를 만드는 과정은 message 함수가 담당하게 됩니다.
Message 함수는 그래프에 존재하는 모든 엣지에 대해서, 특정 엣지 e의 특징과 엣지의 양 끝에 연결된 노드 두 개의 특징을 모두 인자로 받아 메시지 m_e를 생성합니다.
이제 생성된 메시지 m_e는 그래프의 노드들로 passing이 될 준비가 되었습니다.
Passing하는 과정은 update 함수가 담당하게 됩니다.
Update 함수는 그래프에 존재하는 모든 노드들에 대해, 특정 노드의 특징벡터와, 해당 노드에 연결된 엣지들에서 생성된 메시지를 인자로 받아 노드의 특징벡터를 update하게 됩니다.
이 과정이 바로 생성되었던 message를 passing하는 과정으로 볼 수 있으며 이 두 과정을 종합하여 Message-passing이라고 합니다.
GCN은 특정 노드에 대해 해당 노드와, 해당 노드의 이웃 노드들의 특징 벡터를 모두 평균을 내는 AGGREGATE/COMBINE과정을 갖는다고 알고있습니다.
즉 GCN의 message함수는 엣지마다 연결되어있는 노드 특징을 그대로 보존하고, update함수가 각 노드에서 해당 노드특징과 엣지에 연결된 노드특징들의 평균을 내준다고 볼 수 있습니다.
DGL에서는 이러한 message-passing 과정을 직접 원하는 대로 설정할 수 있도록 함수들을 제공하고 있습니다.

In [5]:
class GraphConv(nn.Module):
    def __init__(self, input_dimension, output_dimension):
        super().__init__()
        self.linear = nn.Linear(input_dimension, output_dimension) # message-passing된 노드 특징을 매핑해주는 fully-connected 층
        
    def forward(self, g, node_feat):
        with g.local_scope(): # local_scope내에서 DGL 그래프 객체 g에 수정을 가하더라도, local_scope를 빠져나오면 g의 수정은 사라지도록 함.
            g.ndata['x'] = node_feat # 노드 피쳐를 DGL 함수에서 쓸 수 있도록 DGL 그래프 객체의 ndata 속성에 'x'라는 key로 대응해줌
            g.update_all(dgl.function.copy_u(u='x', out='m'), dgl.function.mean(msg='m', out='h')) # copy_u()라는 message 함수와, mean()이라는 update함수를 이용해 그래프를 업데이트함 (=message-passing을 진행함)
            return self.linear(g.ndata['h']) # 위의 update함수에서 out='h'로 인자를 주었기 때문에 연산의 결과는 그래프 객체의 ndata 속성에 'h'라는 key의 값으로 대응됨. 이를 fully-connected층을 통과시켜 반환함

`GraphConv` 클래스가 정의된다면, GCN 모델을 만드는 것은 GCN1을 만들 때와 거의 같습니다.
이번 모델 클래스는 GCN2로 정의하겠습니다.

In [6]:
class GCN2(nn.Module):
    def __init__(self, node_feat_length, num_hidden, num_classes):
        super().__init__()
        self.conv1 = GraphConv(node_feat_length, num_hidden) # 직접 정의한 GraphConv를 이용해 하나의 GCN 층을 구성할 수 있음.
        self.relu = nn.ReLU()
        self.conv2 = GraphConv(num_hidden, num_classes)
        
    def forward(self, g, node_feat):
        h = self.conv1(g, node_feat)
        h = self.relu(h)
        h = self.conv2(g, h)
        return h

#### (3) 어렵게 구현하기: DGL을 사용하지 않고 모델을 정의

DGL에서는 편리한 함수들을 효율적으로 구현하여 제공해 주지만, 원하는 실험을 하기 위해서는 DGL에서 제공하는 함수를 사용하지 않고 직접 모델을 구현해야 하는 경우도 있습니다.
GCN정도는 DGL의 도움을 받지 않고 구현하여 보는 것이 경험상 좋을 수 있습니다.
조금 더 심화된 내용을 원하신다면 이 구현 방법을 같이 해 보시고, 원치 않으신다면 그냥 넘어가도 좋습니다.
(여기에서 구현되는 GCN을 이해하기 위해서는 본래 논문의 결과 수식인 $Z=\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}X\Theta$ 를 이해하셔야 합니다.)

In [7]:
class GraphConvMM(nn.Module): # MM은 matrix multiplication을 뜻합니다.
    def __init__(self, input_dimension, output_dimension):
        super().__init__()
        self.linear = nn.Linear(input_dimension, output_dimension)
        
    def forward(self, g, node_feat):
        D = torch.sparse_coo_tensor(torch.tensor([[i,i] for i in range(len(node_feat))]).t(), g.in_degrees().float().rsqrt(), g.adjacency_matrix().size())
        A = g.adjacency_matrix()
        X = node_feat
        
        DX = torch.spmm(D, X)
        ADX = torch.spmm(A, DX)
        DADX = torch.spmm(D, ADX)
        Z = self.linear(DADX)
        return Z

위 `GraphConvMM`은 dgl에서 제공하는 함수를 전혀 사용하지 않고, 원래 논문의 수식대로 행렬곱(Matrix multiplication)만을 이용해 GCN layer를 정의하였습니다.
이로부터 GCN 모델을 만드는 것은 GCN1, GCN2와 마찬가지로 어렵지 않습니다.

In [8]:
class GCN3(nn.Module):
    def __init__(self, node_feat_length, num_hidden, num_classes):
        super().__init__()
        self.conv1 = GraphConvMM(node_feat_length, num_hidden) # 직접 정의한 GraphConvMM을 이용해 하나의 GCN 층을 구성할 수 있음.
        self.relu = nn.ReLU()
        self.conv2 = GraphConv(num_hidden, num_classes)
        
    def forward(self, g, node_feat):
        h = self.conv1(g, node_feat)
        h = self.relu(h)
        h = self.conv2(g, h)
        return h

여기까지 세 가지 방법으로 GCN 모델을 구현해 보았습니다.
이제 각 모델이 실제로 원하는 분류 태스크에서 성능이 충분히 나오는지 실험을 해 볼 차례입니다.

### 3. 실험: GCN으로 Pubmed 출판물 분류 학습하기

이번 섹션에서는 정의된 GCN 모델들과 Pubmed 데이터셋으로, 출판물에 해당하는 그래프의 각 노드가 어느 분류에 속하는지 학습을 해 보겠습니다.
이를 위해서 다음과 같은 함수를 정의합니다.

In [9]:
class experiment(object):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
        
    def train(self, g, node_feat, label, train_mask=None, val_mask=None, epochs=50, lr=0.01):
        xent = nn.CrossEntropyLoss() # Loss(Cross entropy)를 정의
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr) # Optimizer(Adam)을 정의
        
        g.add_edges(g.nodes(), g.nodes()) # G에 각 노드별로 스스로에 대한 엣지(self-loop)을 추가하여 AGGREGATE/COMBINE시에 함께 평균이 계산될 수 있도록 함.

        self.model.train() # 모델을 train 상태로 변환
        start_time = time.time()
        for epoch in range(epochs):
            if (epoch+1)%10==0 or epoch==0: print(f'Epoch {epoch+1}')
            logit = self.model(g, node_feat)
            loss = xent(logit, label) if train_mask is None else xent(logit[train_mask], label[train_mask]) # train_mask가 주어진다면 주어진 학습데이터에 대해서만 loss를 계산함.
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_acc = (logit.argmax(1) == label).float().mean() if train_mask is None else (logit[train_mask].argmax(1) == label[train_mask]).float().mean()
            if (epoch+1)%10==0 or epoch==0: print(f'Train accuracy: {train_acc}')
            
            if val_mask is not None:
                val_acc = (logit[val_mask].argmax(1) == label[val_mask]).float().mean()
                if (epoch+1)%10==0 or epoch==0: print(f'Validation accuracy: {val_acc}')
        end_time = time.time()
        print(f'=== Training finished in {(end_time-start_time):.4f} seconds')
            
    def test(self, g, node_feat, label, test_mask=None):
        g.add_edges(g.nodes(), g.nodes())
        self.model.eval()
        with torch.no_grad():
            logit = self.model(g, node_feat)
            test_acc = (logit.argmax(1) == label).float().mean() if test_mask is None else (logit[test_mask].argmax(1) == label[test_mask]).float().mean()
        print(f'Test accuracy: {test_acc}')
        

이제 실험을 할 수 있는 클래스를 정의를 하였으니, 앞서 만든 세 개의 모델로 각각 실험 객체를 만들어 보겠습니다. 또 편의상 실험에 인자로 필요한 g, node_feat, label, train_mask, val_mask, test_mask를 다시한번 정의하겠습니다.

In [10]:
# 세 종류의 모델 객체 생성
model1 = GCN1(node_feat_length=500, num_hidden=64, num_classes=3) # 쉽게 정의된 GCN
model2 = GCN2(node_feat_length=500, num_hidden=64, num_classes=3) # 직접 정의한 GCN
model3 = GCN3(node_feat_length=500, num_hidden=64, num_classes=3) # 어렵게 정의한 GCN (DGL 미사용)

# 세 종류의 실험 객체 생성
experiment1 = experiment(model1)
experiment2 = experiment(model2)
experiment3 = experiment(model3)

# 학습에 필요한 인자 재정의
g, node_feat, label, train_mask, val_mask, test_mask = g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], g.ndata['val_mask'], g.ndata['test_mask']

#### 실험 1: 쉽게 정의한 GCN의 성능
이제 실험 1부터 학습하고, 테스트셋에서의 결과를 확인하겠습니다.

In [11]:
experiment1.train(g, node_feat, label, train_mask, val_mask)
experiment1.test(g, node_feat, label, test_mask)

Epoch 1
Train accuracy: 0.36666667461395264
Validation accuracy: 0.35600000619888306
Epoch 10
Train accuracy: 0.9333333373069763
Validation accuracy: 0.7419999837875366
Epoch 20
Train accuracy: 0.949999988079071
Validation accuracy: 0.7540000081062317
Epoch 30
Train accuracy: 0.9666666388511658
Validation accuracy: 0.7680000066757202
Epoch 40
Train accuracy: 1.0
Validation accuracy: 0.7760000228881836
Epoch 50
Train accuracy: 1.0
Validation accuracy: 0.7760000228881836
=== Training finished in 1.1720 seconds
Test accuracy: 0.7699999809265137


>학습에는 약 1.17초가 소요되었고, 테스트 정확도는 77.0% 가량이 도출되었습니다.
우연히 정답을 맞출 확률이 1/3, 약 33.3%임을 고려하면 의미있는 학습이 되었다고 볼 수 있겠습니다.

#### 실험 2: 직접 정의한 GCN의 성능

실험 2에서도 같은 조건으로 결과를 확인해 볼 수 있습니다.

In [12]:
experiment2.train(g, node_feat, label, train_mask, val_mask)
experiment2.test(g, node_feat, label, test_mask)

Epoch 1
Train accuracy: 0.3333333432674408
Validation accuracy: 0.19599999487400055
Epoch 10
Train accuracy: 0.9333333373069763
Validation accuracy: 0.7379999756813049
Epoch 20
Train accuracy: 0.9333333373069763
Validation accuracy: 0.7440000176429749
Epoch 30
Train accuracy: 0.9666666388511658
Validation accuracy: 0.7540000081062317
Epoch 40
Train accuracy: 0.9833333492279053
Validation accuracy: 0.7699999809265137
Epoch 50
Train accuracy: 0.9833333492279053
Validation accuracy: 0.7680000066757202
=== Training finished in 1.8178 seconds
Test accuracy: 0.7580000162124634


>학습에는 약 1.82초가 소요되었고, 테스트 정확도는 75.8% 가량이 도출되었습니다.
쉽게 정의된 GCN1에 비해 큰 시간이 소요되고, 정확도가 감소하였습니다.

#### 실험 3: 어렵게 정의한 GCN의 성능

실험 3에서 결과를 확인해 보도록 하겠습니다.

In [13]:
experiment3.train(g, node_feat, label, train_mask, val_mask)
experiment3.test(g, node_feat, label, test_mask)

Epoch 1
Train accuracy: 0.3333333432674408
Validation accuracy: 0.41600000858306885
Epoch 10
Train accuracy: 0.949999988079071
Validation accuracy: 0.7319999933242798
Epoch 20
Train accuracy: 0.9666666388511658
Validation accuracy: 0.7440000176429749
Epoch 30
Train accuracy: 0.9666666388511658
Validation accuracy: 0.7480000257492065
Epoch 40
Train accuracy: 0.9833333492279053
Validation accuracy: 0.75
Epoch 50
Train accuracy: 1.0
Validation accuracy: 0.7440000176429749
=== Training finished in 4.3542 seconds
Test accuracy: 0.7360000014305115


>학습에는 약 4.35초가 소요되었고, 테스트 정확도는 73.6% 가량이 도출되었습니다.
세 종류의 GCN 모델 중 가장 시간이 오래 걸리는 것을 볼 수 있습니다.

세 모델의 테스트 정확도가 조금씩 상이하지만, 이는 결과에 영향을 줄 수 있는 하이퍼파라미터(lr, epoch수 등)를 조정하지 않고 확인한 결과이기 때문에 세 모델의 성능이 다르다고 지금 결과만으로 단언할 수는 없습니다.
다만 확실한 것은 가장 쉽게 정의된 GCN1이 가장 속도가 빠르다는 점입니다.
많이 사용되는 모델들은 이렇게 보통 라이브러리에서 제공하는 함수를 사용하는 것이 최적화가 잘 되어 있어 속도가 더 우수한 경우를 종종 볼 수 있습니다.
만약 모델자체에 변형을 가해 실험을 하는 경우가 아니라면, GCN1 처럼 쉽게 모델을 정의하여 사용하는 것이 오히려 빠르고, 정확하게 원하는 결과를 얻을 수 있겠습니다.

### 마치며

이번 시간에는 GNN을 직접 구현해 보는 시간을 가졌습니다.
그 과정에서 

- 그래프 구조 데이터를 파악하고 불러오기
- GNN 모델을 정의하기
    1. DGL에서 제공하는 Layer함수 사용
    2. DGL의 Message-passing을 이용하여 Layer함수를 직접 정의
    3. DGL을 사용하지 않고 논문의 수식을 직접 pytorch로 정의
- GNN 모델로 그래프 구조 데이터를 이용한 노드 분류 학습하기

를 공부할 수 있었습니다.
오늘 시간을 통해 앞으로 다양한 그래프 데이터를 다루고, 원하는 GNN 모델을 구현하여 실험할 수 있는 기반에 조금이나마 도움이 되었기를 바랍니다.
